In [1]:
from sagemaker import get_execution_role
role = get_execution_role()

#Bucket and Folder
bucket = 'project-b-data'
subfolder = 'proj-b/dataset-b01'

In [2]:
import boto3
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']
#Data Set Name
for i in range(5):
    print(contents[i]['Key'])

proj-b/dataset-b01/README.txt
proj-b/dataset-b01/Tst2022-01-04LOBs.txt
proj-b/dataset-b01/Tst2022-01-04tapes.csv
proj-b/dataset-b01/Tst2022-01-05LOBs.txt
proj-b/dataset-b01/Tst2022-01-05tapes.csv


In [3]:
#Generate String Date
import pandas as pd
import time
import re
date_list = pd.date_range(start = "2022-01-04",end = "2022-04-29").tolist()
exact_date = [str(i)[0:10] for i in date_list]   #2022-01-04 format"
print(exact_date[:5])

['2022-01-04', '2022-01-05', '2022-01-06', '2022-01-07', '2022-01-08']


In [5]:
tape_csv = []

total_time = 0
for the_date in exact_date:
    t1 = time.time()
    tape_from_s3 = 's3://{}/{}'.format(bucket,subfolder+'/Tst'+the_date+'tapes.csv')

    try: 
        tape_csv.append(pd.DataFrame(pd.read_csv(tape_from_s3,header = None)))
    except:
        continue
    t2 = time.time()
    total_time += t2 - t1
    print('The {} is finished. Elapsed time is {} seconds.'.format(the_date,t2-t1))

print("Total .csv file Read time: {}".format(total_time))

The 2022-01-04 is finished. Elapsed time is 10.253955602645874 seconds.
The 2022-01-05 is finished. Elapsed time is 0.675506591796875 seconds.
The 2022-01-06 is finished. Elapsed time is 0.5000510215759277 seconds.
The 2022-01-07 is finished. Elapsed time is 0.5559003353118896 seconds.
The 2022-01-10 is finished. Elapsed time is 0.5946023464202881 seconds.
The 2022-01-11 is finished. Elapsed time is 0.6860418319702148 seconds.
The 2022-01-12 is finished. Elapsed time is 0.5395212173461914 seconds.
The 2022-01-13 is finished. Elapsed time is 0.8686573505401611 seconds.
The 2022-01-14 is finished. Elapsed time is 0.5591864585876465 seconds.
The 2022-01-17 is finished. Elapsed time is 0.5260851383209229 seconds.
The 2022-01-18 is finished. Elapsed time is 0.45144057273864746 seconds.
The 2022-01-19 is finished. Elapsed time is 0.6032276153564453 seconds.
The 2022-01-20 is finished. Elapsed time is 0.42289137840270996 seconds.
The 2022-01-21 is finished. Elapsed time is 0.5730395317077637 

In [10]:
#Concatenate all the csv file into one data frame 


full_csv_frame = pd.concat(tape_csv)
print("There are {} rows and {} columns of the full csv frame.".format(full_csv_frame.shape[0],full_csv_frame.shape[1]))
print(full_csv_frame[:5])
full_csv_frame.to_csv("Tape_csv/Full_tape.csv",index = False)

There are 5230364 rows and 11 columns of the full csv frame.
              0          1       2    3                        4   \
0  Tst2022-01-04   Exch0Lit   8.192  203   {'pool_id': 'Exch0Lit'   
1  Tst2022-01-04   Exch0Lit   8.192  199   {'pool_id': 'Exch0Lit'   
2  Tst2022-01-04   Exch0Lit  10.336  196   {'pool_id': 'Exch0Lit'   
3  Tst2022-01-04   Exch0Lit  10.672  196   {'pool_id': 'Exch0Lit'   
4  Tst2022-01-04   Exch0Lit  11.040  201   {'pool_id': 'Exch0Lit'   

                 5                6              7          8   \
0   'type': 'Trade'    'time': 8.192   'price': 203   'qty': 2   
1   'type': 'Trade'    'time': 8.192   'price': 199   'qty': 3   
2   'type': 'Trade'   'time': 10.336   'price': 196   'qty': 3   
3   'type': 'Trade'   'time': 10.672   'price': 196   'qty': 4   
4   'type': 'Trade'    'time': 11.04   'price': 201   'qty': 1   

                 9                  10  
0   'party1': 'S26'   'party2': 'B24'}  
1   'party1': 'S26'   'party2': 'B08'}  
2   

In [12]:
tape = pd.read_csv("Tape_csv/Full_tape.csv")

In [13]:
new_tape = tape.rename(columns = {"0":"Date",
                       "2":"Time","3":"Price","8":"Quantity",
                      "9":"Party1","10":"Party2"}).drop(['1',"4","5","6","7"],axis = 1)
new_tape

,Date,Time,Price,Quantity,Party1,Party2
0,Tst2022-01-04,8.192,203,'qty': 2,'party1': 'S26','party2': 'B24'}
1,Tst2022-01-04,8.192,199,'qty': 3,'party1': 'S26','party2': 'B08'}
2,Tst2022-01-04,10.336,196,'qty': 3,'party1': 'S06','party2': 'B07'}
3,Tst2022-01-04,10.672,196,'qty': 4,'party1': 'S01','party2': 'B01'}
4,Tst2022-01-04,11.040,201,'qty': 1,'party1': 'S12','party2': 'B31'}
...,...,...,...,...,...,...
5230359,Tst2022-04-29,30599.376,134,'qty': 1,'party1': 'S21','party2': 'B07'}
5230360,Tst2022-04-29,30599.520,134,'qty': 1,'party1': 'S03','party2': 'B07'}
5230361,Tst2022-04-29,30599.584,134,'qty': 1,'party1': 'S28','party2': 'B01'}
5230362,Tst2022-04-29,30599.776,134,'qty': 3,'party1': 'S04','party2': 'B01'}


In [14]:

new_tape["Date"] = list(map(lambda x:x[3:],new_tape["Date"]))
new_tape["Quantity"] = list(map(lambda x: float(x[-1]),new_tape["Quantity"]))
new_tape["Party1"] = list(map(lambda x: re.findall(r"S[\d][\d]",x)[0],new_tape["Party1"]))
new_tape["Party2"] = list(map(lambda x: re.findall(r"B[\d][\d]",x)[0],new_tape["Party2"]))
new_tape

,Date,Time,Price,Quantity,Party1,Party2
0,2022-01-04,8.192,203,2.0,S26,B24
1,2022-01-04,8.192,199,3.0,S26,B08
2,2022-01-04,10.336,196,3.0,S06,B07
3,2022-01-04,10.672,196,4.0,S01,B01
4,2022-01-04,11.040,201,1.0,S12,B31
...,...,...,...,...,...,...
5230359,2022-04-29,30599.376,134,1.0,S21,B07
5230360,2022-04-29,30599.520,134,1.0,S03,B07
5230361,2022-04-29,30599.584,134,1.0,S28,B01
5230362,2022-04-29,30599.776,134,3.0,S04,B01


In [15]:
new_tape.iloc[13]["Time"]

16.112000000000002

In [16]:
new_tape["Time"]  = new_tape["Time"].round(decimals = 3)

In [17]:
new_tape.iloc[13]["Time"]

16.112

In [18]:
new_tape.dtypes

Date         object
Time        float64
Price         int64
Quantity    float64
Party1       object
Party2       object
dtype: object

In [19]:
new_tape.to_csv("Tape_csv/Full_tape.csv",index = False)

In [20]:
pd.read_csv("Tape_csv/Full_tape.csv")

,Date,Time,Price,Quantity,Party1,Party2
0,2022-01-04,8.192,203,2.0,S26,B24
1,2022-01-04,8.192,199,3.0,S26,B08
2,2022-01-04,10.336,196,3.0,S06,B07
3,2022-01-04,10.672,196,4.0,S01,B01
4,2022-01-04,11.040,201,1.0,S12,B31
...,...,...,...,...,...,...
5230359,2022-04-29,30599.376,134,1.0,S21,B07
5230360,2022-04-29,30599.520,134,1.0,S03,B07
5230361,2022-04-29,30599.584,134,1.0,S28,B01
5230362,2022-04-29,30599.776,134,3.0,S04,B01


In [23]:
%%time
new_tape.to_csv(f"s3://project-b-data/proj-b/Tape_csv/New_Full_Tape.csv",index = False)

CPU times: user 16.3 s, sys: 378 ms, total: 16.7 s
Wall time: 20.3 s


# LOB

In [4]:
!pip install smart_open

import re
import smart_open as sm

     |████████████████████████████████| 58 kB 8.9 MB/s             


In [5]:
import ast

In [6]:
def Read_LOB_Improved(source):
    LOB_subset_time = []
    LOB_subset_bid = []
    LOB_subset_ask = []
    time_i = -10000
    bid_i = -10000
    ask_i = -10000
    time_change_sign = False
    bid_change_sign = False
    ask_change_sign = False
    
    #Attention: errors = "ignore" for 2022-01-28
    with sm.open(source,"r",encoding="utf-8",errors = "ignore") as f:
        for i,line in enumerate(f):
            if i% 1000000 == 0:
                print(f"This is {i//1000000}th 1000000 row")
    
            #Time
            if line.strip() == '"time",':
                time_i = i 
                time_change_sign = True
                continue        
                
            if i == (time_i +1) and time_change_sign == True:
                time_number = re.findall(r"[\d]+[\.]?[\d]*",line)[0]
                #print(f"Time Number {time_number} Appended")
                time_change_sign = False
                LOB_subset_time.append(time_number)
                continue



            #Bid
            if line.strip() == '"bid",':
                bid_i = i
                bid_change_sign = True
                bid_string = ""
                continue
            if i == (bid_i + 1) and line.strip() == "[]" and bid_change_sign == True:
                #print("Bid None Appended")
                bid_change_sign = False
                LOB_subset_bid.append(None)
                continue          
            if i == (bid_i + 1) and line.strip() == "[" and bid_change_sign == True:
                bid_string += line.strip()
                continue
            elif line.strip() != "]" and bid_change_sign == True:
                bid_string += line.strip()
                continue
            elif line.strip() == "]" and bid_change_sign == True:
                bid_string += "]]"
                #Use for detect the fault line
                try:
                    #Use for 2022-01-14
                    LOB_subset_bid.append(ast.literal_eval(bid_string.strip().replace("M","").replace(" ","").replace(chr(0),"")))
                except ValueError:
                    print(f'This is {i}th row, {bid_string.strip().replace("M","").replace(" ","").replace(chr(0),"")} wrong')
                          
                except:
                    print(f'Something Wrong {i} rows')
                    
                    
                #print(f"Bid {bid_string} Appended")
                bid_change_sign = False
                continue

            #Ask
            if line.strip() == '"ask",':
                ask_i = i
                ask_change_sign = True
                ask_string = ""
                continue       
            if i == (ask_i + 1) and line.strip() == "[]" and ask_change_sign == True:
                LOB_subset_ask.append(None)
                #print("Ask None Appended")
                ask_change_sign = False
                continue       
            if i == (ask_i + 1) and line.strip() == "[" and ask_change_sign == True:
                ask_string += line.strip()
                continue
            elif line.strip() != "]" and ask_change_sign == True:
                ask_string += line.strip()
                continue
            elif line.strip() == "]" and ask_change_sign == True:
                ask_string += "]]"
                #2022-01-28
                try:
                    LOB_subset_ask.append(ast.literal_eval(ask_string.strip().replace("M","").replace(" ","").replace(chr(0),"")))
                except ValueError:
                    print(f'This is {i}th row, {ask_string.strip().replace("M","").replace(" ","").replace(chr(0),"")} wrong')
                          
                except:
                    print(f'Something Wrong {i} rows')
                                          
                #print(f"Ask {ask_string} Appended")
                ask_change_sign = False
                continue
        f.close()
        
        return [LOB_subset_time,LOB_subset_bid,LOB_subset_ask]
        
def Create_LOB_df_Improved(df_element,the_date):
    
    
    LOB_df = pd.DataFrame(df_element).T
    LOB_df['Date'] = the_date
    
    LOB_df = LOB_df.rename(columns={0: 'Time', 1: 'Bid',2: 'Ask'})
                          
    #Directly to S3
    LOB_df.to_csv(f's3://project-b-data/proj-b/LOB_csv/{the_date}-LOB.csv',index = False)
    print(f"The Shape of {the_date} LOB is {LOB_df.shape}")
    
    return LOB_df
        
        
    

In [ ]:
%%time
#The Shape of 2022-01-04 LOB is (1912500, 4)
#2022-01-04 Finished, LOB Shape: (1912500, 4)
#The 2022-01-04 is finished. Elapsed time is 524.5722577571869 seconds.


#Something Wrong 155491955 rows 2022-01-28


#"s3://project-b-data/proj-b/dataset-b01/Tst2022-01-04LOBs.txt" format
for the_date in exact_date[1:]:
    t1 = time.time()
    LOB_from_s3 = 's3://{}/{}'.format(bucket,subfolder+'/Tst'+the_date+'LOBs.txt')
    

    try:
        print(f"Start Read {the_date} LOB file.")
        Create_LOB_df_Improved(Read_LOB_Improved(LOB_from_s3),the_date)
    except:
        print(f"There is no {the_date} LOB file.")
        continue
        
    t2 = time.time()
    print('The {} is finished. Elapsed time is {} seconds.'.format(the_date,t2-t1))


Start Read 2022-01-05 LOB file.
This is 0th 1000000 row
This is 1th 1000000 row
This is 2th 1000000 row
This is 3th 1000000 row
This is 4th 1000000 row
This is 5th 1000000 row
This is 6th 1000000 row
This is 7th 1000000 row
This is 8th 1000000 row
This is 9th 1000000 row
This is 10th 1000000 row
This is 11th 1000000 row
This is 12th 1000000 row
This is 13th 1000000 row
This is 14th 1000000 row
This is 15th 1000000 row
This is 16th 1000000 row
This is 17th 1000000 row
This is 18th 1000000 row
This is 19th 1000000 row
This is 20th 1000000 row
This is 21th 1000000 row
This is 22th 1000000 row
This is 23th 1000000 row
This is 24th 1000000 row
This is 25th 1000000 row
This is 26th 1000000 row
This is 27th 1000000 row
This is 28th 1000000 row
This is 29th 1000000 row
This is 30th 1000000 row
This is 31th 1000000 row
This is 32th 1000000 row
This is 33th 1000000 row
This is 34th 1000000 row
This is 35th 1000000 row
This is 36th 1000000 row
This is 37th 1000000 row
This is 38th 1000000 row
Thi

In [7]:
pd.read_csv(f"s3://project-b-data/proj-b/LOB_csv/2022-01-04-LOB.csv")

,Time,Bid,Ask,Date
0,0.000,NaN,NaN,2022-01-04
1,0.016,NaN,NaN,2022-01-04
2,0.032,NaN,NaN,2022-01-04
3,0.048,NaN,NaN,2022-01-04
4,0.064,NaN,NaN,2022-01-04
...,...,...,...,...
1912495,30599.920,"[[220, 4], [211, 1], [175, 4], [170, 5], [161,...","[[236, 2], [411, 5], [436, 1], [659, 1], [725,...",2022-01-04
1912496,30599.936,"[[220, 4], [211, 1], [175, 4], [170, 5], [161,...","[[236, 2], [411, 5], [436, 1], [659, 1], [725,...",2022-01-04
1912497,30599.952,"[[220, 4], [211, 1], [175, 4], [170, 5], [161,...","[[236, 2], [411, 5], [436, 1], [659, 1], [725,...",2022-01-04
1912498,30599.968,"[[220, 4], [211, 1], [175, 4], [170, 5], [161,...","[[236, 2], [411, 5], [436, 1], [611, 1], [725,...",2022-01-04


In [8]:
pd.read_csv(f"s3://project-b-data/proj-b/LOB_csv/2022-01-14-LOB.csv")

,Time,Bid,Ask,Date
0,0.000,NaN,NaN,2022-01-14
1,0.016,NaN,NaN,2022-01-14
2,0.032,NaN,NaN,2022-01-14
3,0.048,NaN,NaN,2022-01-14
4,0.064,NaN,NaN,2022-01-14
...,...,...,...,...
1912487,30599.920,"[[227, 2], [225, 2], [221, 3], [188, 1], [184,...","[[230, 2], [418, 4], [428, 4], [456, 4], [534,...",2022-01-14
1912488,30599.936,"[[227, 2], [225, 2], [221, 3], [188, 1], [184,...","[[230, 2], [418, 4], [428, 4], [456, 4], [534,...",2022-01-14
1912489,30599.952,"[[227, 2], [225, 2], [221, 3], [188, 1], [184,...","[[230, 2], [418, 4], [428, 4], [456, 4], [534,...",2022-01-14
1912490,30599.968,"[[227, 2], [225, 2], [221, 3], [188, 1], [184,...","[[230, 2], [418, 4], [428, 4], [456, 4], [534,...",2022-01-14


In [9]:
pd.read_csv(f"s3://project-b-data/proj-b/LOB_csv/2022-01-28-LOB.csv")

,Time,Bid,Ask,Date
0,0.000,NaN,NaN,2022-01-28
1,0.016,NaN,NaN,2022-01-28
2,0.032,NaN,NaN,2022-01-28
3,0.048,NaN,NaN,2022-01-28
4,0.064,NaN,NaN,2022-01-28
...,...,...,...,...
1912487,30599.920,"[[202, 3], [199, 3], [188, 4], [173, 7], [154,...","[[213, 4], [219, 1], [220, 5], [330, 2], [377,...",2022-01-28
1912488,30599.936,"[[202, 3], [199, 3], [188, 4], [173, 7], [154,...","[[213, 4], [219, 1], [220, 5], [330, 2], [377,...",2022-01-28
1912489,30599.952,"[[202, 3], [199, 3], [188, 4], [173, 4], [154,...","[[213, 4], [219, 1], [220, 5], [330, 2], [377,...",2022-01-28
1912490,30599.968,"[[202, 3], [199, 3], [188, 4], [173, 4], [154,...","[[213, 4], [219, 1], [220, 5], [330, 2], [377,...",2022-01-28


In [10]:
exact_date[25:]

['2022-01-29',
 '2022-01-30',
 '2022-01-31',
 '2022-02-01',
 '2022-02-02',
 '2022-02-03',
 '2022-02-04',
 '2022-02-05',
 '2022-02-06',
 '2022-02-07',
 '2022-02-08',
 '2022-02-09',
 '2022-02-10',
 '2022-02-11',
 '2022-02-12',
 '2022-02-13',
 '2022-02-14',
 '2022-02-15',
 '2022-02-16',
 '2022-02-17',
 '2022-02-18',
 '2022-02-19',
 '2022-02-20',
 '2022-02-21',
 '2022-02-22',
 '2022-02-23',
 '2022-02-24',
 '2022-02-25',
 '2022-02-26',
 '2022-02-27',
 '2022-02-28',
 '2022-03-01',
 '2022-03-02',
 '2022-03-03',
 '2022-03-04',
 '2022-03-05',
 '2022-03-06',
 '2022-03-07',
 '2022-03-08',
 '2022-03-09',
 '2022-03-10',
 '2022-03-11',
 '2022-03-12',
 '2022-03-13',
 '2022-03-14',
 '2022-03-15',
 '2022-03-16',
 '2022-03-17',
 '2022-03-18',
 '2022-03-19',
 '2022-03-20',
 '2022-03-21',
 '2022-03-22',
 '2022-03-23',
 '2022-03-24',
 '2022-03-25',
 '2022-03-26',
 '2022-03-27',
 '2022-03-28',
 '2022-03-29',
 '2022-03-30',
 '2022-03-31',
 '2022-04-01',
 '2022-04-02',
 '2022-04-03',
 '2022-04-04',
 '2022-04-

In [ ]:
%%time
#The Shape of 2022-01-04 LOB is (1912500, 4)
#2022-01-04 Finished, LOB Shape: (1912500, 4)
#The 2022-01-04 is finished. Elapsed time is 524.5722577571869 seconds.





#"s3://project-b-data/proj-b/dataset-b01/Tst2022-01-04LOBs.txt" format
for the_date in exact_date[25:]:
    t1 = time.time()
    LOB_from_s3 = 's3://{}/{}'.format(bucket,subfolder+'/Tst'+the_date+'LOBs.txt')
    

    try:
        print(f"Start Read {the_date} LOB file.")
        Create_LOB_df_Improved(Read_LOB_Improved(LOB_from_s3),the_date)
    except:
        print(f"There is no {the_date} LOB file.")
        continue
        
    t2 = time.time()
    print('The {} is finished. Elapsed time is {} seconds.'.format(the_date,t2-t1))

Start Read 2022-01-29 LOB file.
There is no 2022-01-29 LOB file.
Start Read 2022-01-30 LOB file.
There is no 2022-01-30 LOB file.
Start Read 2022-01-31 LOB file.
This is 0th 1000000 row
This is 1th 1000000 row
This is 2th 1000000 row
This is 3th 1000000 row
This is 4th 1000000 row
This is 5th 1000000 row
This is 6th 1000000 row
This is 7th 1000000 row
This is 8th 1000000 row
This is 9th 1000000 row
This is 10th 1000000 row
This is 11th 1000000 row
This is 12th 1000000 row
This is 13th 1000000 row
This is 14th 1000000 row
This is 15th 1000000 row
This is 16th 1000000 row
This is 17th 1000000 row
This is 18th 1000000 row
This is 19th 1000000 row
This is 20th 1000000 row
This is 21th 1000000 row
This is 22th 1000000 row
This is 23th 1000000 row
This is 24th 1000000 row
This is 25th 1000000 row
This is 26th 1000000 row
This is 27th 1000000 row
This is 28th 1000000 row
This is 29th 1000000 row
This is 30th 1000000 row
This is 31th 1000000 row
This is 32th 1000000 row
This is 33th 1000000 ro

In [11]:
exact_date[63:]

['2022-03-08',
 '2022-03-09',
 '2022-03-10',
 '2022-03-11',
 '2022-03-12',
 '2022-03-13',
 '2022-03-14',
 '2022-03-15',
 '2022-03-16',
 '2022-03-17',
 '2022-03-18',
 '2022-03-19',
 '2022-03-20',
 '2022-03-21',
 '2022-03-22',
 '2022-03-23',
 '2022-03-24',
 '2022-03-25',
 '2022-03-26',
 '2022-03-27',
 '2022-03-28',
 '2022-03-29',
 '2022-03-30',
 '2022-03-31',
 '2022-04-01',
 '2022-04-02',
 '2022-04-03',
 '2022-04-04',
 '2022-04-05',
 '2022-04-06',
 '2022-04-07',
 '2022-04-08',
 '2022-04-09',
 '2022-04-10',
 '2022-04-11',
 '2022-04-12',
 '2022-04-13',
 '2022-04-14',
 '2022-04-15',
 '2022-04-16',
 '2022-04-17',
 '2022-04-18',
 '2022-04-19',
 '2022-04-20',
 '2022-04-21',
 '2022-04-22',
 '2022-04-23',
 '2022-04-24',
 '2022-04-25',
 '2022-04-26',
 '2022-04-27',
 '2022-04-28',
 '2022-04-29']

In [12]:
%%time
#The Shape of 2022-01-04 LOB is (1912500, 4)
#2022-01-04 Finished, LOB Shape: (1912500, 4)
#The 2022-01-04 is finished. Elapsed time is 524.5722577571869 seconds.





#"s3://project-b-data/proj-b/dataset-b01/Tst2022-01-04LOBs.txt" format
for the_date in exact_date[63:]:
    t1 = time.time()
    LOB_from_s3 = 's3://{}/{}'.format(bucket,subfolder+'/Tst'+the_date+'LOBs.txt')
    

    try:
        print(f"Start Read {the_date} LOB file.")
        Create_LOB_df_Improved(Read_LOB_Improved(LOB_from_s3),the_date)
    except:
        print(f"There is no {the_date} LOB file.")
        continue
        
    t2 = time.time()
    print('The {} is finished. Elapsed time is {} seconds.'.format(the_date,t2-t1))

Start Read 2022-03-08 LOB file.
This is 0th 1000000 row
This is 1th 1000000 row
This is 2th 1000000 row
This is 3th 1000000 row
This is 4th 1000000 row
This is 5th 1000000 row
This is 6th 1000000 row
This is 7th 1000000 row
This is 8th 1000000 row
This is 9th 1000000 row
This is 10th 1000000 row
This is 11th 1000000 row
This is 12th 1000000 row
This is 13th 1000000 row
This is 14th 1000000 row
This is 15th 1000000 row
This is 16th 1000000 row
This is 17th 1000000 row
This is 18th 1000000 row
This is 19th 1000000 row
This is 20th 1000000 row
This is 21th 1000000 row
This is 22th 1000000 row
This is 23th 1000000 row
This is 24th 1000000 row
This is 25th 1000000 row
This is 26th 1000000 row
This is 27th 1000000 row
This is 28th 1000000 row
This is 29th 1000000 row
This is 30th 1000000 row
This is 31th 1000000 row
This is 32th 1000000 row
This is 33th 1000000 row
This is 34th 1000000 row
This is 35th 1000000 row
This is 36th 1000000 row
This is 37th 1000000 row
This is 38th 1000000 row
Thi

# Combine

In [13]:
new_tape = pd.read_csv('s3://project-b-data/proj-b/Tape_csv/New_Full_Tape.csv')

In [14]:
new_tape

,Date,Time,Price,Quantity,Party1,Party2
0,2022-01-04,8.192,203,2.0,S26,B24
1,2022-01-04,8.192,199,3.0,S26,B08
2,2022-01-04,10.336,196,3.0,S06,B07
3,2022-01-04,10.672,196,4.0,S01,B01
4,2022-01-04,11.040,201,1.0,S12,B31
...,...,...,...,...,...,...
5230359,2022-04-29,30599.376,134,1.0,S21,B07
5230360,2022-04-29,30599.520,134,1.0,S03,B07
5230361,2022-04-29,30599.584,134,1.0,S28,B01
5230362,2022-04-29,30599.776,134,3.0,S04,B01


In [29]:
type(new_tape.loc[0]["Party2"])

str

In [15]:
LOB = pd.DataFrame()
print(LOB)

Empty DataFrame
Columns: []
Index: []


In [16]:
%%time
total_file_number = 0
for date in exact_date:
    try:
        LOB = LOB.append(pd.read_csv(f's3://project-b-data/proj-b/LOB_csv/{date}-LOB.csv'))
        print(f"{date} Finished, LOB Shape: {LOB.shape}")
        total_file_number += 1
    except FileNotFoundError:
        print(f"No file in {date}.")
    except:
        print(f"Something Wrong in {date}!")

2022-01-04 Finished, LOB Shape: (1912500, 4)
2022-01-05 Finished, LOB Shape: (3825000, 4)
2022-01-06 Finished, LOB Shape: (5737500, 4)
2022-01-07 Finished, LOB Shape: (7650000, 4)
No file in 2022-01-08.
No file in 2022-01-09.
2022-01-10 Finished, LOB Shape: (9562500, 4)
2022-01-11 Finished, LOB Shape: (11475000, 4)
2022-01-12 Finished, LOB Shape: (13387500, 4)
2022-01-13 Finished, LOB Shape: (15300000, 4)
2022-01-14 Finished, LOB Shape: (17212492, 4)
No file in 2022-01-15.
No file in 2022-01-16.
2022-01-17 Finished, LOB Shape: (19124992, 4)
2022-01-18 Finished, LOB Shape: (21037492, 4)
2022-01-19 Finished, LOB Shape: (22949992, 4)
2022-01-20 Finished, LOB Shape: (24862492, 4)
2022-01-21 Finished, LOB Shape: (26774992, 4)
No file in 2022-01-22.
No file in 2022-01-23.
2022-01-24 Finished, LOB Shape: (28687492, 4)
2022-01-25 Finished, LOB Shape: (30599992, 4)
2022-01-26 Finished, LOB Shape: (32512492, 4)
2022-01-27 Finished, LOB Shape: (34424992, 4)
2022-01-28 Finished, LOB Shape: (363374

In [17]:
print(total_file_number)

48


In [18]:
LOB.shape

(91799980, 4)

In [19]:
LOB.head()

,Time,Bid,Ask,Date
0,0.000,NaN,NaN,2022-01-04
1,0.016,NaN,NaN,2022-01-04
2,0.032,NaN,NaN,2022-01-04
3,0.048,NaN,NaN,2022-01-04
4,0.064,NaN,NaN,2022-01-04


In [20]:
LOB.to_csv(f"s3://project-b-data/proj-b/LOB_csv/Full_LOB/New_Full_LOB.csv",index = False)

In [44]:
type(LOB.iloc[0]["Date"])

str

In [46]:
LOB.iloc[512]

Time                                                8.192
Bid     [[203, 2], [199, 3], [195, 4], [187, 5], [181,...
Ask     [[207, 4], [211, 4], [214, 2], [220, 5], [226,...
Date                                           2022-01-04
Name: 512, dtype: object

In [47]:
#Can't use merge because of memory error, Should use chunk.
Tape_with_LOB = pd.DataFrame()


In [48]:
%%time


for date in list(new_tape["Date"].unique()):
    new_tape_date = new_tape.loc[new_tape["Date"] == date]
    LOB_date = LOB.loc[LOB["Date"] == date]
    new_df = pd.merge(new_tape_date, LOB_date,  how='left', left_on=['Date','Time'], right_on = ['Date','Time'])
    Tape_with_LOB = Tape_with_LOB.append(new_df)
    print(f"{date} Finished.")

2022-01-04 Finished.
2022-01-05 Finished.
2022-01-06 Finished.
2022-01-07 Finished.
2022-01-10 Finished.
2022-01-11 Finished.
2022-01-12 Finished.
2022-01-13 Finished.
2022-01-14 Finished.
2022-01-17 Finished.
2022-01-18 Finished.
2022-01-19 Finished.
2022-01-20 Finished.
2022-01-21 Finished.
2022-01-24 Finished.
2022-01-25 Finished.
2022-01-26 Finished.
2022-01-27 Finished.
2022-01-28 Finished.
2022-01-31 Finished.
2022-02-01 Finished.
2022-02-02 Finished.
2022-02-03 Finished.
2022-02-04 Finished.
2022-02-07 Finished.
2022-02-08 Finished.
2022-02-09 Finished.
2022-02-10 Finished.
2022-02-11 Finished.
2022-02-14 Finished.
2022-02-15 Finished.
2022-02-16 Finished.
2022-02-17 Finished.
2022-02-18 Finished.
2022-02-21 Finished.
2022-02-22 Finished.
2022-02-23 Finished.
2022-02-24 Finished.
2022-02-25 Finished.
2022-03-01 Finished.
2022-03-02 Finished.
2022-03-03 Finished.
2022-03-04 Finished.
2022-03-07 Finished.
2022-03-08 Finished.
2022-03-09 Finished.
2022-03-10 Finished.
2022-03-11 Fi

In [49]:
Tape_with_LOB

,Date,Time,Price,Quantity,Party1,Party2,Bid,Ask
0,2022-01-04,8.192,203,2.0,S26,B24,"[[203, 2], [199, 3], [195, 4], [187, 5], [181,...","[[207, 4], [211, 4], [214, 2], [220, 5], [226,..."
1,2022-01-04,8.192,199,3.0,S26,B08,"[[203, 2], [199, 3], [195, 4], [187, 5], [181,...","[[207, 4], [211, 4], [214, 2], [220, 5], [226,..."
2,2022-01-04,10.336,196,3.0,S06,B07,"[[196, 3], [195, 4], [187, 8], [182, 1], [156,...","[[197, 4], [198, 3], [201, 1], [202, 4], [209,..."
3,2022-01-04,10.672,196,4.0,S01,B01,"[[195, 4], [187, 5], [183, 3], [182, 1], [156,...","[[196, 4], [201, 1], [203, 4], [209, 9], [214,..."
4,2022-01-04,11.040,201,1.0,S12,B31,"[[192, 4], [187, 5], [182, 1], [176, 5], [168,...","[[201, 1], [203, 4], [205, 5], [209, 4], [214,..."
...,...,...,...,...,...,...,...,...
64888,2022-04-29,30599.376,134,1.0,S21,B07,"[[134, 8], [133, 9], [132, 3], [121, 5], [93, ...","[[158, 1], [206, 2], [329, 3], [418, 5], [479,..."
64889,2022-04-29,30599.520,134,1.0,S03,B07,"[[134, 5], [133, 9], [132, 3], [121, 5], [93, ...","[[158, 1], [329, 3], [418, 5], [479, 4], [502,..."
64890,2022-04-29,30599.584,134,1.0,S28,B01,"[[134, 4], [133, 9], [132, 3], [121, 5], [93, ...","[[158, 1], [329, 3], [418, 5], [479, 4], [502,..."
64891,2022-04-29,30599.776,134,3.0,S04,B01,"[[134, 3], [133, 9], [132, 3], [121, 5], [93, ...","[[158, 1], [329, 3], [418, 5], [479, 4], [502,..."


In [54]:
Tape_with_LOB.loc[Tape_with_LOB["Date"] == "2022-02-21"].head(30)

,Date,Time,Price,Quantity,Party1,Party2,Bid,Ask
0,2022-02-21,3.312,232,1.0,S13,B23,"[[197, 1], [188, 1], [176, 1], [175, 5], [173,...","[[232, 3], [233, 4], [614, 3]]"
1,2022-02-21,4.704,210,4.0,S05,B28,"[[200, 1], [199, 5], [197, 1], [193, 6], [176,...","[[210, 15], [229, 2], [235, 5], [686, 3]]"
2,2022-02-21,7.024,210,4.0,S25,B11,"[[209, 2], [208, 5], [206, 1], [205, 1], [200,...","[[210, 14], [229, 2], [232, 3], [235, 5], [343..."
3,2022-02-21,7.024,210,1.0,S23,B11,"[[209, 2], [208, 5], [206, 1], [205, 1], [200,...","[[210, 14], [229, 2], [232, 3], [235, 5], [343..."
4,2022-02-21,7.248,210,1.0,S23,B01,"[[209, 2], [208, 5], [206, 1], [205, 1], [200,...","[[210, 9], [229, 2], [232, 3], [235, 5], [343,..."
5,2022-02-21,7.328,210,2.0,S00,B14,"[[209, 2], [208, 5], [206, 1], [200, 5], [198,...","[[210, 8], [229, 2], [232, 3], [235, 5], [343,..."
6,2022-02-21,7.888,210,2.0,S00,B10,"[[209, 3], [208, 5], [206, 1], [199, 5], [198,...","[[210, 6], [229, 2], [232, 3], [235, 5], [263,..."
7,2022-02-21,7.904,210,1.0,S01,B31,"[[209, 3], [208, 5], [206, 1], [199, 5], [198,...","[[210, 4], [229, 2], [232, 3], [235, 5], [263,..."
8,2022-02-21,7.904,210,2.0,S11,B31,"[[209, 3], [208, 5], [206, 1], [199, 5], [198,...","[[210, 4], [229, 2], [232, 3], [235, 5], [263,..."
9,2022-02-21,8.048,210,1.0,S11,B16,"[[209, 3], [208, 5], [206, 1], [199, 5], [198,...","[[210, 1], [229, 2], [232, 3], [235, 5], [263,..."


In [55]:
Tape_with_LOB.loc[Tape_with_LOB["Date"] == "2022-02-21"].iloc[19]["Time"]

17.055999999999997

In [56]:
Tape_with_LOB.loc[Tape_with_LOB["Date"] == "2022-01-04"].head(30)

,Date,Time,Price,Quantity,Party1,Party2,Bid,Ask
0,2022-01-04,8.192,203,2.0,S26,B24,"[[203, 2], [199, 3], [195, 4], [187, 5], [181,...","[[207, 4], [211, 4], [214, 2], [220, 5], [226,..."
1,2022-01-04,8.192,199,3.0,S26,B08,"[[203, 2], [199, 3], [195, 4], [187, 5], [181,...","[[207, 4], [211, 4], [214, 2], [220, 5], [226,..."
2,2022-01-04,10.336,196,3.0,S06,B07,"[[196, 3], [195, 4], [187, 8], [182, 1], [156,...","[[197, 4], [198, 3], [201, 1], [202, 4], [209,..."
3,2022-01-04,10.672,196,4.0,S01,B01,"[[195, 4], [187, 5], [183, 3], [182, 1], [156,...","[[196, 4], [201, 1], [203, 4], [209, 9], [214,..."
4,2022-01-04,11.040,201,1.0,S12,B31,"[[192, 4], [187, 5], [182, 1], [176, 5], [168,...","[[201, 1], [203, 4], [205, 5], [209, 4], [214,..."
5,2022-01-04,11.040,203,2.0,S09,B31,"[[192, 4], [187, 5], [182, 1], [176, 5], [168,...","[[201, 1], [203, 4], [205, 5], [209, 4], [214,..."
6,2022-01-04,13.008,197,3.0,S07,B27,"[[197, 3], [196, 2], [191, 7], [188, 5], [187,...","[[200, 6], [203, 5], [205, 2], [206, 2], [207,..."
7,2022-01-04,13.008,196,2.0,S07,B03,"[[197, 3], [196, 2], [191, 7], [188, 5], [187,...","[[200, 6], [203, 5], [205, 2], [206, 2], [207,..."
8,2022-01-04,14.080,193,1.0,S31,B23,"[[193, 4], [192, 4], [189, 5], [187, 1], [186,...","[[196, 2], [197, 1], [198, 6], [199, 5], [202,..."
9,2022-01-04,14.672,195,1.0,S23,B06,"[[194, 4], [193, 3], [189, 5], [183, 3], [170,...","[[195, 1], [196, 2], [197, 5], [198, 1], [199,..."


In [61]:
Tape_with_LOB.loc[Tape_with_LOB["Date"] == "2022-01-28"].tail(100)

,Date,Time,Price,Quantity,Party1,Party2,Bid,Ask
64496,2022-01-28,30554.656,204,2.0,S22,B10,"[[204, 11], [203, 7], [195, 3], [158, 5], [157...","[[211, 5], [212, 5], [213, 4], [348, 1], [360,..."
64497,2022-01-28,30555.600,204,1.0,S06,B10,"[[204, 10], [203, 7], [158, 5], [157, 3], [120...","[[208, 5], [210, 1], [212, 5], [348, 1], [365,..."
64498,2022-01-28,30555.792,204,2.0,S01,B06,"[[204, 9], [203, 7], [157, 3], [120, 3], [63, ...","[[208, 5], [210, 1], [212, 5], [365, 4], [372,..."
64499,2022-01-28,30556.480,204,1.0,S09,B06,"[[204, 7], [203, 7], [153, 3], [139, 3], [97, ...","[[205, 5], [206, 4], [207, 1], [208, 5], [218,..."
64500,2022-01-28,30556.544,204,1.0,S08,B06,"[[204, 6], [203, 7], [153, 3], [139, 3], [97, ...","[[205, 5], [206, 4], [208, 5], [218, 1], [365,..."
...,...,...,...,...,...,...,...,...
64591,2022-01-28,30599.456,204,2.0,S13,B06,"[[204, 2], [203, 1], [202, 7], [199, 3], [173,...","[[219, 1], [220, 5], [330, 2], [377, 4], [389,..."
64592,2022-01-28,30599.456,203,1.0,S13,B27,"[[204, 2], [203, 1], [202, 7], [199, 3], [173,...","[[219, 1], [220, 5], [330, 2], [377, 4], [389,..."
64593,2022-01-28,30599.728,202,1.0,S15,B09,"[[202, 7], [199, 3], [188, 4], [173, 7], [154,...","[[213, 4], [219, 1], [220, 5], [330, 2], [377,..."
64594,2022-01-28,30599.728,202,1.0,S15,B16,"[[202, 7], [199, 3], [188, 4], [173, 7], [154,...","[[213, 4], [219, 1], [220, 5], [330, 2], [377,..."


In [57]:
Tape_with_LOB.to_csv(f"s3://project-b-data/proj-b/Tape_with_LOB/Tape_with_LOB.csv",index = False)

In [59]:
%%time
import numpy as np
for date in list(Tape_with_LOB["Date"].unique()):
    nan_number = np.sum(list(map(lambda x:pd.isna(x),Tape_with_LOB.loc[Tape_with_LOB["Date"] == date]["Bid"])))
    print(f"{date} nan Number {nan_number}")

2022-01-04 nan Number 0
2022-01-05 nan Number 0
2022-01-06 nan Number 3
2022-01-07 nan Number 0
2022-01-10 nan Number 0
2022-01-11 nan Number 11
2022-01-12 nan Number 11
2022-01-13 nan Number 4
2022-01-14 nan Number 3
2022-01-17 nan Number 3
2022-01-18 nan Number 2
2022-01-19 nan Number 2
2022-01-20 nan Number 2
2022-01-21 nan Number 6
2022-01-24 nan Number 0
2022-01-25 nan Number 4
2022-01-26 nan Number 6
2022-01-27 nan Number 2
2022-01-28 nan Number 1
2022-01-31 nan Number 0
2022-02-01 nan Number 16
2022-02-02 nan Number 3
2022-02-03 nan Number 2
2022-02-04 nan Number 0
2022-02-07 nan Number 0
2022-02-08 nan Number 0
2022-02-09 nan Number 0
2022-02-10 nan Number 1
2022-02-11 nan Number 1
2022-02-14 nan Number 0
2022-02-15 nan Number 1
2022-02-16 nan Number 1
2022-02-17 nan Number 0
2022-02-18 nan Number 4
2022-02-21 nan Number 0
2022-02-22 nan Number 6
2022-02-23 nan Number 3
2022-02-24 nan Number 1
2022-02-25 nan Number 5
2022-03-01 nan Number 9
2022-03-02 nan Number 0
2022-03-03 na

In [62]:
for date in list(Tape_with_LOB["Date"].unique()):
    nan_number = np.sum(list(map(lambda x:pd.isna(x),Tape_with_LOB.loc[Tape_with_LOB["Date"] == date]["Ask"])))
    print(f"{date} nan Number {nan_number}")

2022-01-04 nan Number 0
2022-01-05 nan Number 0
2022-01-06 nan Number 0
2022-01-07 nan Number 1
2022-01-10 nan Number 0
2022-01-11 nan Number 0
2022-01-12 nan Number 0
2022-01-13 nan Number 0
2022-01-14 nan Number 0
2022-01-17 nan Number 0
2022-01-18 nan Number 1
2022-01-19 nan Number 0
2022-01-20 nan Number 0
2022-01-21 nan Number 2
2022-01-24 nan Number 0
2022-01-25 nan Number 0
2022-01-26 nan Number 0
2022-01-27 nan Number 0
2022-01-28 nan Number 0
2022-01-31 nan Number 0
2022-02-01 nan Number 0
2022-02-02 nan Number 0
2022-02-03 nan Number 0
2022-02-04 nan Number 2
2022-02-07 nan Number 0
2022-02-08 nan Number 0
2022-02-09 nan Number 0
2022-02-10 nan Number 0
2022-02-11 nan Number 0
2022-02-14 nan Number 0
2022-02-15 nan Number 0
2022-02-16 nan Number 0
2022-02-17 nan Number 0
2022-02-18 nan Number 1
2022-02-21 nan Number 0
2022-02-22 nan Number 0
2022-02-23 nan Number 0
2022-02-24 nan Number 0
2022-02-25 nan Number 0
2022-03-01 nan Number 0
2022-03-02 nan Number 0
2022-03-03 nan N

In [73]:
17.056/0.016

1066.0

In [78]:
LOB.loc[LOB["Date"] == "2022-02-21"].iloc[1066]["Time"]

17.055999999999997

In [82]:
17.055%0.5

0.054999999999999716

In [90]:
LOB["Time"]  = LOB["Time"].round(decimals = 3)

In [91]:
LOB.loc[LOB["Date"] == "2022-02-21"].iloc[1066]["Time"]

17.056

In [92]:
%%time


Tape_with_LOB = pd.DataFrame()
for date in list(new_tape["Date"].unique()):
    new_tape_date = new_tape.loc[new_tape["Date"] == date]
    LOB_date = LOB.loc[LOB["Date"] == date]
    new_df = pd.merge(new_tape_date, LOB_date,  how='left', left_on=['Date','Time'], right_on = ['Date','Time'])
    Tape_with_LOB = Tape_with_LOB.append(new_df)
    print(f"{date} Finished.")

2022-01-04 Finished.
2022-01-05 Finished.
2022-01-06 Finished.
2022-01-07 Finished.
2022-01-10 Finished.
2022-01-11 Finished.
2022-01-12 Finished.
2022-01-13 Finished.
2022-01-14 Finished.
2022-01-17 Finished.
2022-01-18 Finished.
2022-01-19 Finished.
2022-01-20 Finished.
2022-01-21 Finished.
2022-01-24 Finished.
2022-01-25 Finished.
2022-01-26 Finished.
2022-01-27 Finished.
2022-01-28 Finished.
2022-01-31 Finished.
2022-02-01 Finished.
2022-02-02 Finished.
2022-02-03 Finished.
2022-02-04 Finished.
2022-02-07 Finished.
2022-02-08 Finished.
2022-02-09 Finished.
2022-02-10 Finished.
2022-02-11 Finished.
2022-02-14 Finished.
2022-02-15 Finished.
2022-02-16 Finished.
2022-02-17 Finished.
2022-02-18 Finished.
2022-02-21 Finished.
2022-02-22 Finished.
2022-02-23 Finished.
2022-02-24 Finished.
2022-02-25 Finished.
2022-03-01 Finished.
2022-03-02 Finished.
2022-03-03 Finished.
2022-03-04 Finished.
2022-03-07 Finished.
2022-03-08 Finished.
2022-03-09 Finished.
2022-03-10 Finished.
2022-03-11 Fi

In [93]:
%%time
import numpy as np
for date in list(Tape_with_LOB["Date"].unique()):
    nan_number = np.sum(list(map(lambda x:pd.isna(x),Tape_with_LOB.loc[Tape_with_LOB["Date"] == date]["Bid"])))
    print(f"{date} nan Number {nan_number}")

2022-01-04 nan Number 16614
2022-01-05 nan Number 16181
2022-01-06 nan Number 16610
2022-01-07 nan Number 16583
2022-01-10 nan Number 16596
2022-01-11 nan Number 16556
2022-01-12 nan Number 16466
2022-01-13 nan Number 16546
2022-01-14 nan Number 16335
2022-01-17 nan Number 16780
2022-01-18 nan Number 16497
2022-01-19 nan Number 16343
2022-01-20 nan Number 16630
2022-01-21 nan Number 16513
2022-01-24 nan Number 16654
2022-01-25 nan Number 16463
2022-01-26 nan Number 16753
2022-01-27 nan Number 16491
2022-01-28 nan Number 16686
2022-01-31 nan Number 16848
2022-02-01 nan Number 16466
2022-02-02 nan Number 16563
2022-02-03 nan Number 16624
2022-02-04 nan Number 16697
2022-02-07 nan Number 16769
2022-02-08 nan Number 16706
2022-02-09 nan Number 16500
2022-02-10 nan Number 16298
2022-02-11 nan Number 16271
2022-02-14 nan Number 16364
2022-02-15 nan Number 16829
2022-02-16 nan Number 16412
2022-02-17 nan Number 16543
2022-02-18 nan Number 16638
2022-02-21 nan Number 16347
2022-02-22 nan Numbe

In [94]:
for date in list(Tape_with_LOB["Date"].unique()):
    nan_number = np.sum(list(map(lambda x:pd.isna(x),Tape_with_LOB.loc[Tape_with_LOB["Date"] == date]["Ask"])))
    print(f"{date} nan Number {nan_number}")

2022-01-04 nan Number 16614
2022-01-05 nan Number 16181
2022-01-06 nan Number 16607
2022-01-07 nan Number 16584
2022-01-10 nan Number 16596
2022-01-11 nan Number 16549
2022-01-12 nan Number 16455
2022-01-13 nan Number 16542
2022-01-14 nan Number 16333
2022-01-17 nan Number 16777
2022-01-18 nan Number 16496
2022-01-19 nan Number 16342
2022-01-20 nan Number 16629
2022-01-21 nan Number 16509
2022-01-24 nan Number 16654
2022-01-25 nan Number 16461
2022-01-26 nan Number 16751
2022-01-27 nan Number 16489
2022-01-28 nan Number 16685
2022-01-31 nan Number 16848
2022-02-01 nan Number 16457
2022-02-02 nan Number 16560
2022-02-03 nan Number 16623
2022-02-04 nan Number 16699
2022-02-07 nan Number 16769
2022-02-08 nan Number 16706
2022-02-09 nan Number 16500
2022-02-10 nan Number 16297
2022-02-11 nan Number 16270
2022-02-14 nan Number 16364
2022-02-15 nan Number 16828
2022-02-16 nan Number 16411
2022-02-17 nan Number 16543
2022-02-18 nan Number 16639
2022-02-21 nan Number 16347
2022-02-22 nan Numbe

In [96]:
Tape_with_LOB.head(30)

,Date,Time,Price,Quantity,Party1,Party2,Bid,Ask
0,2022-01-04,8.192,203,2.0,S26,B24,"[[203, 2], [199, 3], [195, 4], [187, 5], [181,...","[[207, 4], [211, 4], [214, 2], [220, 5], [226,..."
1,2022-01-04,8.192,199,3.0,S26,B08,"[[203, 2], [199, 3], [195, 4], [187, 5], [181,...","[[207, 4], [211, 4], [214, 2], [220, 5], [226,..."
2,2022-01-04,10.336,196,3.0,S06,B07,NaN,NaN
3,2022-01-04,10.672,196,4.0,S01,B01,"[[195, 4], [187, 5], [183, 3], [182, 1], [156,...","[[196, 4], [201, 1], [203, 4], [209, 9], [214,..."
4,2022-01-04,11.040,201,1.0,S12,B31,"[[192, 4], [187, 5], [182, 1], [176, 5], [168,...","[[201, 1], [203, 4], [205, 5], [209, 4], [214,..."
5,2022-01-04,11.040,203,2.0,S09,B31,"[[192, 4], [187, 5], [182, 1], [176, 5], [168,...","[[201, 1], [203, 4], [205, 5], [209, 4], [214,..."
6,2022-01-04,13.008,197,3.0,S07,B27,"[[197, 3], [196, 2], [191, 7], [188, 5], [187,...","[[200, 6], [203, 5], [205, 2], [206, 2], [207,..."
7,2022-01-04,13.008,196,2.0,S07,B03,"[[197, 3], [196, 2], [191, 7], [188, 5], [187,...","[[200, 6], [203, 5], [205, 2], [206, 2], [207,..."
8,2022-01-04,14.080,193,1.0,S31,B23,"[[193, 4], [192, 4], [189, 5], [187, 1], [186,...","[[196, 2], [197, 1], [198, 6], [199, 5], [202,..."
9,2022-01-04,14.672,195,1.0,S23,B06,"[[194, 4], [193, 3], [189, 5], [183, 3], [170,...","[[195, 1], [196, 2], [197, 5], [198, 1], [199,..."


In [97]:
10.336/0.016

646.0

In [99]:
LOB.loc[LOB["Date"] == "2022-01-04"].iloc[646]["Time"]

10.336

In [103]:
new_tape.loc[new_tape["Date"] == "2022-01-04"].iloc[2]["Time"]

10.335999999999999

In [104]:
new_tape["Time"] = new_tape["Time"].round(decimals = 3)

In [105]:
new_tape.loc[new_tape["Date"] == "2022-01-04"].iloc[2]["Time"]

10.336

In [106]:
%%time


Tape_with_LOB = pd.DataFrame()
for date in list(new_tape["Date"].unique()):
    new_tape_date = new_tape.loc[new_tape["Date"] == date]
    LOB_date = LOB.loc[LOB["Date"] == date]
    new_df = pd.merge(new_tape_date, LOB_date,  how='left', left_on=['Date','Time'], right_on = ['Date','Time'])
    Tape_with_LOB = Tape_with_LOB.append(new_df)
    print(f"{date} Finished.")

2022-01-04 Finished.
2022-01-05 Finished.
2022-01-06 Finished.
2022-01-07 Finished.
2022-01-10 Finished.
2022-01-11 Finished.
2022-01-12 Finished.
2022-01-13 Finished.
2022-01-14 Finished.
2022-01-17 Finished.
2022-01-18 Finished.
2022-01-19 Finished.
2022-01-20 Finished.
2022-01-21 Finished.
2022-01-24 Finished.
2022-01-25 Finished.
2022-01-26 Finished.
2022-01-27 Finished.
2022-01-28 Finished.
2022-01-31 Finished.
2022-02-01 Finished.
2022-02-02 Finished.
2022-02-03 Finished.
2022-02-04 Finished.
2022-02-07 Finished.
2022-02-08 Finished.
2022-02-09 Finished.
2022-02-10 Finished.
2022-02-11 Finished.
2022-02-14 Finished.
2022-02-15 Finished.
2022-02-16 Finished.
2022-02-17 Finished.
2022-02-18 Finished.
2022-02-21 Finished.
2022-02-22 Finished.
2022-02-23 Finished.
2022-02-24 Finished.
2022-02-25 Finished.
2022-03-01 Finished.
2022-03-02 Finished.
2022-03-03 Finished.
2022-03-04 Finished.
2022-03-07 Finished.
2022-03-08 Finished.
2022-03-09 Finished.
2022-03-10 Finished.
2022-03-11 Fi

In [108]:
Tape_with_LOB

,Date,Time,Price,Quantity,Party1,Party2,Bid,Ask
0,2022-01-04,8.192,203,2.0,S26,B24,"[[203, 2], [199, 3], [195, 4], [187, 5], [181,...","[[207, 4], [211, 4], [214, 2], [220, 5], [226,..."
1,2022-01-04,8.192,199,3.0,S26,B08,"[[203, 2], [199, 3], [195, 4], [187, 5], [181,...","[[207, 4], [211, 4], [214, 2], [220, 5], [226,..."
2,2022-01-04,10.336,196,3.0,S06,B07,"[[196, 3], [195, 4], [187, 8], [182, 1], [156,...","[[197, 4], [198, 3], [201, 1], [202, 4], [209,..."
3,2022-01-04,10.672,196,4.0,S01,B01,"[[195, 4], [187, 5], [183, 3], [182, 1], [156,...","[[196, 4], [201, 1], [203, 4], [209, 9], [214,..."
4,2022-01-04,11.040,201,1.0,S12,B31,"[[192, 4], [187, 5], [182, 1], [176, 5], [168,...","[[201, 1], [203, 4], [205, 5], [209, 4], [214,..."
...,...,...,...,...,...,...,...,...
64888,2022-04-29,30599.376,134,1.0,S21,B07,"[[134, 8], [133, 9], [132, 3], [121, 5], [93, ...","[[158, 1], [206, 2], [329, 3], [418, 5], [479,..."
64889,2022-04-29,30599.520,134,1.0,S03,B07,"[[134, 5], [133, 9], [132, 3], [121, 5], [93, ...","[[158, 1], [329, 3], [418, 5], [479, 4], [502,..."
64890,2022-04-29,30599.584,134,1.0,S28,B01,"[[134, 4], [133, 9], [132, 3], [121, 5], [93, ...","[[158, 1], [329, 3], [418, 5], [479, 4], [502,..."
64891,2022-04-29,30599.776,134,3.0,S04,B01,"[[134, 3], [133, 9], [132, 3], [121, 5], [93, ...","[[158, 1], [329, 3], [418, 5], [479, 4], [502,..."


In [109]:
%%time
import numpy as np
for date in list(Tape_with_LOB["Date"].unique()):
    nan_number = np.sum(list(map(lambda x:pd.isna(x),Tape_with_LOB.loc[Tape_with_LOB["Date"] == date]["Bid"])))
    print(f"{date} nan Number {nan_number}")
    
    
    
'''
2022-01-04 nan Number 0
2022-01-05 nan Number 0
2022-01-06 nan Number 3
2022-01-07 nan Number 0
2022-01-10 nan Number 0
2022-01-11 nan Number 11
2022-01-12 nan Number 11
2022-01-13 nan Number 4
2022-01-14 nan Number 3
2022-01-17 nan Number 3
2022-01-18 nan Number 2
2022-01-19 nan Number 2
2022-01-20 nan Number 2
2022-01-21 nan Number 6
2022-01-24 nan Number 0
2022-01-25 nan Number 4
2022-01-26 nan Number 6
2022-01-27 nan Number 2
2022-01-28 nan Number 1
2022-01-31 nan Number 0
2022-02-01 nan Number 16
2022-02-02 nan Number 3
2022-02-03 nan Number 2
2022-02-04 nan Number 0
2022-02-07 nan Number 0
2022-02-08 nan Number 0
2022-02-09 nan Number 0
2022-02-10 nan Number 1
2022-02-11 nan Number 1
2022-02-14 nan Number 0
2022-02-15 nan Number 1
2022-02-16 nan Number 1
2022-02-17 nan Number 0
2022-02-18 nan Number 4
2022-02-21 nan Number 0
2022-02-22 nan Number 6
2022-02-23 nan Number 3
2022-02-24 nan Number 1
2022-02-25 nan Number 5
2022-03-01 nan Number 9
2022-03-02 nan Number 0
2022-03-03 nan Number 2
2022-03-04 nan Number 4
2022-03-07 nan Number 64394
2022-03-08 nan Number 2
2022-03-09 nan Number 2
2022-03-10 nan Number 4
2022-03-11 nan Number 64542
2022-03-14 nan Number 64823
2022-03-15 nan Number 64179
2022-03-16 nan Number 64473
2022-03-17 nan Number 64612
2022-03-18 nan Number 64730
2022-03-21 nan Number 64376
2022-03-22 nan Number 64332
2022-03-23 nan Number 64448
2022-03-24 nan Number 64367
2022-03-25 nan Number 64639
2022-03-28 nan Number 64219
2022-03-29 nan Number 64773
2022-03-30 nan Number 64532
2022-03-31 nan Number 64327
2022-04-01 nan Number 64863
2022-04-04 nan Number 64784
2022-04-05 nan Number 64738
2022-04-06 nan Number 64811
2022-04-07 nan Number 64463
2022-04-08 nan Number 64732
2022-04-11 nan Number 64535
2022-04-12 nan Number 64756
2022-04-13 nan Number 64680
2022-04-14 nan Number 64631
2022-04-19 nan Number 64802
2022-04-20 nan Number 64544
2022-04-21 nan Number 64863
2022-04-22 nan Number 64572
2022-04-25 nan Number 64604
2022-04-26 nan Number 64765
2022-04-27 nan Number 64582
2022-04-28 nan Number 64717
2022-04-29 nan Number 14



'''

2022-01-04 nan Number 0
2022-01-05 nan Number 0
2022-01-06 nan Number 3
2022-01-07 nan Number 0
2022-01-10 nan Number 0
2022-01-11 nan Number 11
2022-01-12 nan Number 11
2022-01-13 nan Number 4
2022-01-14 nan Number 3
2022-01-17 nan Number 3
2022-01-18 nan Number 2
2022-01-19 nan Number 2
2022-01-20 nan Number 2
2022-01-21 nan Number 6
2022-01-24 nan Number 0
2022-01-25 nan Number 4
2022-01-26 nan Number 6
2022-01-27 nan Number 2
2022-01-28 nan Number 1
2022-01-31 nan Number 0
2022-02-01 nan Number 16
2022-02-02 nan Number 3
2022-02-03 nan Number 2
2022-02-04 nan Number 0
2022-02-07 nan Number 0
2022-02-08 nan Number 0
2022-02-09 nan Number 0
2022-02-10 nan Number 1
2022-02-11 nan Number 1
2022-02-14 nan Number 0
2022-02-15 nan Number 1
2022-02-16 nan Number 1
2022-02-17 nan Number 0
2022-02-18 nan Number 4
2022-02-21 nan Number 0
2022-02-22 nan Number 6
2022-02-23 nan Number 3
2022-02-24 nan Number 1
2022-02-25 nan Number 5
2022-03-01 nan Number 9
2022-03-02 nan Number 0
2022-03-03 na

In [110]:
for date in list(Tape_with_LOB["Date"].unique()):
    nan_number = np.sum(list(map(lambda x:pd.isna(x),Tape_with_LOB.loc[Tape_with_LOB["Date"] == date]["Ask"])))
    print(f"{date} nan Number {nan_number}")
    
'''
2022-01-04 nan Number 0
2022-01-05 nan Number 0
2022-01-06 nan Number 0
2022-01-07 nan Number 1
2022-01-10 nan Number 0
2022-01-11 nan Number 0
2022-01-12 nan Number 0
2022-01-13 nan Number 0
2022-01-14 nan Number 0
2022-01-17 nan Number 0
2022-01-18 nan Number 1
2022-01-19 nan Number 0
2022-01-20 nan Number 0
2022-01-21 nan Number 2
2022-01-24 nan Number 0
2022-01-25 nan Number 0
2022-01-26 nan Number 0
2022-01-27 nan Number 0
2022-01-28 nan Number 0
2022-01-31 nan Number 0
2022-02-01 nan Number 0
2022-02-02 nan Number 0
2022-02-03 nan Number 0
2022-02-04 nan Number 2
2022-02-07 nan Number 0
2022-02-08 nan Number 0
2022-02-09 nan Number 0
2022-02-10 nan Number 0
2022-02-11 nan Number 0
2022-02-14 nan Number 0
2022-02-15 nan Number 0
2022-02-16 nan Number 0
2022-02-17 nan Number 0
2022-02-18 nan Number 1
2022-02-21 nan Number 0
2022-02-22 nan Number 0
2022-02-23 nan Number 0
2022-02-24 nan Number 0
2022-02-25 nan Number 0
2022-03-01 nan Number 0
2022-03-02 nan Number 0
2022-03-03 nan Number 0
2022-03-04 nan Number 0
2022-03-07 nan Number 64394
2022-03-08 nan Number 0
2022-03-09 nan Number 0
2022-03-10 nan Number 0
2022-03-11 nan Number 64542
2022-03-14 nan Number 64823
2022-03-15 nan Number 64179
2022-03-16 nan Number 64473
2022-03-17 nan Number 64612
2022-03-18 nan Number 64730
2022-03-21 nan Number 64376
2022-03-22 nan Number 64332
2022-03-23 nan Number 64448
2022-03-24 nan Number 64367
2022-03-25 nan Number 64639
2022-03-28 nan Number 64219
2022-03-29 nan Number 64773
2022-03-30 nan Number 64532
2022-03-31 nan Number 64327
2022-04-01 nan Number 64863
2022-04-04 nan Number 64784
2022-04-05 nan Number 64738
2022-04-06 nan Number 64811
2022-04-07 nan Number 64463
2022-04-08 nan Number 64732
2022-04-11 nan Number 64535
2022-04-12 nan Number 64756
2022-04-13 nan Number 64680
2022-04-14 nan Number 64631
2022-04-19 nan Number 64802
2022-04-20 nan Number 64544
2022-04-21 nan Number 64863
2022-04-22 nan Number 64572
2022-04-25 nan Number 64604
2022-04-26 nan Number 64765
2022-04-27 nan Number 64582
2022-04-28 nan Number 64717
2022-04-29 nan Number 0

'''

2022-01-04 nan Number 0
2022-01-05 nan Number 0
2022-01-06 nan Number 0
2022-01-07 nan Number 1
2022-01-10 nan Number 0
2022-01-11 nan Number 0
2022-01-12 nan Number 0
2022-01-13 nan Number 0
2022-01-14 nan Number 0
2022-01-17 nan Number 0
2022-01-18 nan Number 1
2022-01-19 nan Number 0
2022-01-20 nan Number 0
2022-01-21 nan Number 2
2022-01-24 nan Number 0
2022-01-25 nan Number 0
2022-01-26 nan Number 0
2022-01-27 nan Number 0
2022-01-28 nan Number 0
2022-01-31 nan Number 0
2022-02-01 nan Number 0
2022-02-02 nan Number 0
2022-02-03 nan Number 0
2022-02-04 nan Number 2
2022-02-07 nan Number 0
2022-02-08 nan Number 0
2022-02-09 nan Number 0
2022-02-10 nan Number 0
2022-02-11 nan Number 0
2022-02-14 nan Number 0
2022-02-15 nan Number 0
2022-02-16 nan Number 0
2022-02-17 nan Number 0
2022-02-18 nan Number 1
2022-02-21 nan Number 0
2022-02-22 nan Number 0
2022-02-23 nan Number 0
2022-02-24 nan Number 0
2022-02-25 nan Number 0
2022-03-01 nan Number 0
2022-03-02 nan Number 0
2022-03-03 nan N

In [111]:
new_tape.loc[new_tape["Date"] == "2022-02-21"].iloc[19]["Time"]

17.056

In [112]:
new_tape.loc[new_tape["Date"] == "2022-01-04"].iloc[2]["Time"]

10.336

In [113]:
new_tape = pd.read_csv('s3://project-b-data/proj-b/Tape_csv/New_Full_Tape.csv')

In [114]:
new_tape.loc[new_tape["Date"] == "2022-02-21"].iloc[19]["Time"]

17.055999999999997

In [115]:
new_tape.loc[new_tape["Date"] == "2022-01-04"].iloc[2]["Time"]

10.335999999999999

In [116]:
1/2

0.5

In [117]:
LOB.head()

,Time,Bid,Ask,Date
0,0.000,NaN,NaN,2022-01-04
1,0.016,NaN,NaN,2022-01-04
2,0.032,NaN,NaN,2022-01-04
3,0.048,NaN,NaN,2022-01-04
4,0.064,NaN,NaN,2022-01-04


In [118]:
%%time
import numpy as np
for date in list(Tape_with_LOB["Date"].unique()):
    nan_number = np.sum(list(map(lambda x:pd.isna(x),Tape_with_LOB.loc[Tape_with_LOB["Date"] == date]["Bid"])))
    print(f"{date} nan Number {nan_number}")

2022-01-04 nan Number 0
2022-01-05 nan Number 0
2022-01-06 nan Number 3
2022-01-07 nan Number 0
2022-01-10 nan Number 0
2022-01-11 nan Number 11
2022-01-12 nan Number 11
2022-01-13 nan Number 4
2022-01-14 nan Number 3
2022-01-17 nan Number 3
2022-01-18 nan Number 2
2022-01-19 nan Number 2
2022-01-20 nan Number 2
2022-01-21 nan Number 6
2022-01-24 nan Number 0
2022-01-25 nan Number 4
2022-01-26 nan Number 6
2022-01-27 nan Number 2
2022-01-28 nan Number 1
2022-01-31 nan Number 0
2022-02-01 nan Number 16
2022-02-02 nan Number 3
2022-02-03 nan Number 2
2022-02-04 nan Number 0
2022-02-07 nan Number 0
2022-02-08 nan Number 0
2022-02-09 nan Number 0
2022-02-10 nan Number 1
2022-02-11 nan Number 1
2022-02-14 nan Number 0
2022-02-15 nan Number 1
2022-02-16 nan Number 1
2022-02-17 nan Number 0
2022-02-18 nan Number 4
2022-02-21 nan Number 0
2022-02-22 nan Number 6
2022-02-23 nan Number 3
2022-02-24 nan Number 1
2022-02-25 nan Number 5
2022-03-01 nan Number 9
2022-03-02 nan Number 0
2022-03-03 na

## Redo

In [119]:
new_tape

,Date,Time,Price,Quantity,Party1,Party2
0,2022-01-04,8.192,203,2.0,S26,B24
1,2022-01-04,8.192,199,3.0,S26,B08
2,2022-01-04,10.336,196,3.0,S06,B07
3,2022-01-04,10.672,196,4.0,S01,B01
4,2022-01-04,11.040,201,1.0,S12,B31
...,...,...,...,...,...,...
5230359,2022-04-29,30599.376,134,1.0,S21,B07
5230360,2022-04-29,30599.520,134,1.0,S03,B07
5230361,2022-04-29,30599.584,134,1.0,S28,B01
5230362,2022-04-29,30599.776,134,3.0,S04,B01


In [120]:
new_tape["Time"] = new_tape["Time"].round(decimals = 3)

In [121]:
new_tape

,Date,Time,Price,Quantity,Party1,Party2
0,2022-01-04,8.192,203,2.0,S26,B24
1,2022-01-04,8.192,199,3.0,S26,B08
2,2022-01-04,10.336,196,3.0,S06,B07
3,2022-01-04,10.672,196,4.0,S01,B01
4,2022-01-04,11.040,201,1.0,S12,B31
...,...,...,...,...,...,...
5230359,2022-04-29,30599.376,134,1.0,S21,B07
5230360,2022-04-29,30599.520,134,1.0,S03,B07
5230361,2022-04-29,30599.584,134,1.0,S28,B01
5230362,2022-04-29,30599.776,134,3.0,S04,B01


In [122]:
LOB.head()

,Time,Bid,Ask,Date
0,0.000,NaN,NaN,2022-01-04
1,0.016,NaN,NaN,2022-01-04
2,0.032,NaN,NaN,2022-01-04
3,0.048,NaN,NaN,2022-01-04
4,0.064,NaN,NaN,2022-01-04


In [123]:
LOB["Time"] = LOB["Time"].round(decimals = 3)

In [124]:
LOB.head()

,Time,Bid,Ask,Date
0,0.000,NaN,NaN,2022-01-04
1,0.016,NaN,NaN,2022-01-04
2,0.032,NaN,NaN,2022-01-04
3,0.048,NaN,NaN,2022-01-04
4,0.064,NaN,NaN,2022-01-04


In [125]:
%%time


Tape_with_LOB = pd.DataFrame()
for date in list(new_tape["Date"].unique()):
    new_tape_date = new_tape.loc[new_tape["Date"] == date]
    LOB_date = LOB.loc[LOB["Date"] == date]
    new_df = pd.merge(new_tape_date, LOB_date,  how='left', left_on=['Date','Time'], right_on = ['Date','Time'])
    Tape_with_LOB = Tape_with_LOB.append(new_df)
    print(f"{date} Finished.")

2022-01-04 Finished.
2022-01-05 Finished.
2022-01-06 Finished.
2022-01-07 Finished.
2022-01-10 Finished.
2022-01-11 Finished.
2022-01-12 Finished.
2022-01-13 Finished.
2022-01-14 Finished.
2022-01-17 Finished.
2022-01-18 Finished.
2022-01-19 Finished.
2022-01-20 Finished.
2022-01-21 Finished.
2022-01-24 Finished.
2022-01-25 Finished.
2022-01-26 Finished.
2022-01-27 Finished.
2022-01-28 Finished.
2022-01-31 Finished.
2022-02-01 Finished.
2022-02-02 Finished.
2022-02-03 Finished.
2022-02-04 Finished.
2022-02-07 Finished.
2022-02-08 Finished.
2022-02-09 Finished.
2022-02-10 Finished.
2022-02-11 Finished.
2022-02-14 Finished.
2022-02-15 Finished.
2022-02-16 Finished.
2022-02-17 Finished.
2022-02-18 Finished.
2022-02-21 Finished.
2022-02-22 Finished.
2022-02-23 Finished.
2022-02-24 Finished.
2022-02-25 Finished.
2022-03-01 Finished.
2022-03-02 Finished.
2022-03-03 Finished.
2022-03-04 Finished.
2022-03-07 Finished.
2022-03-08 Finished.
2022-03-09 Finished.
2022-03-10 Finished.
2022-03-11 Fi

In [126]:
Tape_with_LOB

,Date,Time,Price,Quantity,Party1,Party2,Bid,Ask
0,2022-01-04,8.192,203,2.0,S26,B24,"[[203, 2], [199, 3], [195, 4], [187, 5], [181,...","[[207, 4], [211, 4], [214, 2], [220, 5], [226,..."
1,2022-01-04,8.192,199,3.0,S26,B08,"[[203, 2], [199, 3], [195, 4], [187, 5], [181,...","[[207, 4], [211, 4], [214, 2], [220, 5], [226,..."
2,2022-01-04,10.336,196,3.0,S06,B07,"[[196, 3], [195, 4], [187, 8], [182, 1], [156,...","[[197, 4], [198, 3], [201, 1], [202, 4], [209,..."
3,2022-01-04,10.672,196,4.0,S01,B01,"[[195, 4], [187, 5], [183, 3], [182, 1], [156,...","[[196, 4], [201, 1], [203, 4], [209, 9], [214,..."
4,2022-01-04,11.040,201,1.0,S12,B31,"[[192, 4], [187, 5], [182, 1], [176, 5], [168,...","[[201, 1], [203, 4], [205, 5], [209, 4], [214,..."
...,...,...,...,...,...,...,...,...
64888,2022-04-29,30599.376,134,1.0,S21,B07,"[[134, 8], [133, 9], [132, 3], [121, 5], [93, ...","[[158, 1], [206, 2], [329, 3], [418, 5], [479,..."
64889,2022-04-29,30599.520,134,1.0,S03,B07,"[[134, 5], [133, 9], [132, 3], [121, 5], [93, ...","[[158, 1], [329, 3], [418, 5], [479, 4], [502,..."
64890,2022-04-29,30599.584,134,1.0,S28,B01,"[[134, 4], [133, 9], [132, 3], [121, 5], [93, ...","[[158, 1], [329, 3], [418, 5], [479, 4], [502,..."
64891,2022-04-29,30599.776,134,3.0,S04,B01,"[[134, 3], [133, 9], [132, 3], [121, 5], [93, ...","[[158, 1], [329, 3], [418, 5], [479, 4], [502,..."


In [127]:
%%time
import numpy as np
for date in list(Tape_with_LOB["Date"].unique()):
    nan_number = np.sum(list(map(lambda x:pd.isna(x),Tape_with_LOB.loc[Tape_with_LOB["Date"] == date]["Bid"])))
    print(f"{date} nan Number {nan_number}")

2022-01-04 nan Number 0
2022-01-05 nan Number 0
2022-01-06 nan Number 3
2022-01-07 nan Number 0
2022-01-10 nan Number 0
2022-01-11 nan Number 11
2022-01-12 nan Number 11
2022-01-13 nan Number 4
2022-01-14 nan Number 3
2022-01-17 nan Number 3
2022-01-18 nan Number 2
2022-01-19 nan Number 2
2022-01-20 nan Number 2
2022-01-21 nan Number 6
2022-01-24 nan Number 0
2022-01-25 nan Number 4
2022-01-26 nan Number 6
2022-01-27 nan Number 2
2022-01-28 nan Number 1
2022-01-31 nan Number 0
2022-02-01 nan Number 16
2022-02-02 nan Number 3
2022-02-03 nan Number 2
2022-02-04 nan Number 0
2022-02-07 nan Number 0
2022-02-08 nan Number 0
2022-02-09 nan Number 0
2022-02-10 nan Number 1
2022-02-11 nan Number 1
2022-02-14 nan Number 0
2022-02-15 nan Number 1
2022-02-16 nan Number 1
2022-02-17 nan Number 0
2022-02-18 nan Number 4
2022-02-21 nan Number 0
2022-02-22 nan Number 6
2022-02-23 nan Number 3
2022-02-24 nan Number 1
2022-02-25 nan Number 5
2022-03-01 nan Number 9
2022-03-02 nan Number 0
2022-03-03 na

In [128]:
for date in list(Tape_with_LOB["Date"].unique()):
    nan_number = np.sum(list(map(lambda x:pd.isna(x),Tape_with_LOB.loc[Tape_with_LOB["Date"] == date]["Ask"])))
    print(f"{date} nan Number {nan_number}")

2022-01-04 nan Number 0
2022-01-05 nan Number 0
2022-01-06 nan Number 0
2022-01-07 nan Number 1
2022-01-10 nan Number 0
2022-01-11 nan Number 0
2022-01-12 nan Number 0
2022-01-13 nan Number 0
2022-01-14 nan Number 0
2022-01-17 nan Number 0
2022-01-18 nan Number 1
2022-01-19 nan Number 0
2022-01-20 nan Number 0
2022-01-21 nan Number 2
2022-01-24 nan Number 0
2022-01-25 nan Number 0
2022-01-26 nan Number 0
2022-01-27 nan Number 0
2022-01-28 nan Number 0
2022-01-31 nan Number 0
2022-02-01 nan Number 0
2022-02-02 nan Number 0
2022-02-03 nan Number 0
2022-02-04 nan Number 2
2022-02-07 nan Number 0
2022-02-08 nan Number 0
2022-02-09 nan Number 0
2022-02-10 nan Number 0
2022-02-11 nan Number 0
2022-02-14 nan Number 0
2022-02-15 nan Number 0
2022-02-16 nan Number 0
2022-02-17 nan Number 0
2022-02-18 nan Number 1
2022-02-21 nan Number 0
2022-02-22 nan Number 0
2022-02-23 nan Number 0
2022-02-24 nan Number 0
2022-02-25 nan Number 0
2022-03-01 nan Number 0
2022-03-02 nan Number 0
2022-03-03 nan N

In [129]:
Tape_with_LOB.to_csv(f"s3://project-b-data/proj-b/Tape_with_LOB/Tape_with_LOB1.csv",index = False)

In [130]:
LOB.to_csv(f"s3://project-b-data/proj-b/LOB_csv/Full_LOB/New_Full_LOB1.csv",index = False)